# PDS - TP2 Punto 6
### Blasi - Reyes - Sosa Lüchter

Imports

In [ ]:
import numpy as np
import math
from numpy.fft import fft
from numpy.fft import ifft
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Definir señal $x_{2}[n]$.

In [ ]:
# Variable independiente 'n'
L = 1  # Duración [s]
fs = 44.1 * 10 ** 3  # Sample rate
n = np.arange(0, L, 1 / fs)
N = len(n)


# Señal x2[n]
f2 = 10 * 10 ** 3
u2 = 0.2
sigma2 = 0.05
w2 = 2 * np.pi * f2
sub_arg = -((n - u2) ** 2) / (2 * sigma2 ** 2)
x2 = np.cos(w2 * n) * np.e ** sub_arg

Se busca comparar la precisión de dos métodos numéricos para resolver ecuaciónes diferenciales, método de diferencias finitas y método de diferenciación en frecuencia. En primer lugar se define la derivada analítica de la señal $x_{2}[n]$. Luego, se calcula la derivada según los métodos numéricos y se comparan los resultados.

Derivada analítica.

In [ ]:
x2p = -w2 * np.sin(w2*n) * np.e**sub_arg - (2*(n-u2)*np.cos(w2*n)*np.e**sub_arg)/(2*sigma2**2)

Derivada por método de diferencias finitas.

In [ ]:
# Algoritmo según:
# Steven Brunton & Nathan Kutz - Data-Driven Science and Engineering Machine Learning, Dynamical Systems, and Control
# - Cambridge UP -2019

dx = 1/fs
x2p_dif = np.zeros(N)
for i in range(N-1):
    x2p_dif[i] = (x2[i+1]-x2[i])/dx
x2p_dif[-1] = x2p_dif[-2]

Derivada por método de diferenciación en frecuencia.

In [ ]:
# Algoritmo según:
# Steven G. Johnson - Notes on FFT-based differentiation - MIT Applied Mathematics

# Paso 1
X2 = fft(x2)
X2 = X2[:N]
k = np.arange(0, N)

# Paso 2
if ((N/2) % 2) == 0:
    X2a = X2[:int(N/2)] * k[:int(N/2)] * 2j*np.pi/L
    X2b = [0]
    X2c = X2[int(N/2)+1:] * (k[int(N/2)+1:]-N) * 2j*np.pi/L
    X2p_fourier = np.concatenate((X2a, X2b, X2c))
else:
    X2a = X2[:math.ceil(N/2)] * k[:math.ceil(N/2)] * 2j*np.pi/L
    X2b = X2[math.ceil(N/2):] * (k[math.ceil(N/2):]-N) * 2j*np.pi/L
    X2p_fourier = np.concatenate((X2a, X2b))

# Paso 3
x2p_fourier = np.real(ifft(X2p_fourier))

Primero se grafican las señales completas sin acotar.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=n, y=x2p, name='Analítico'))
fig.add_trace(go.Scatter(x=n, y=x2p_dif, name='Dif. finitas'))
fig.add_trace(go.Scatter(x=n, y=x2p_fourier, name='Fourier'))
fig.update_layout(
    title="x2'[n] según metodo de diferenciación - vista completa",
    xaxis_title='Tiempo [s]',
    yaxis_title='Amplitud')
fig.show()

Como se puede ver (se pueden activar/desactivar las señales individuales desde la leyenda), por lo menos a simple vista, la derivada resultante del método de diferenciación por frecuencia es igual a la derivada analítica, mientras que la derivada por el método de diferencias finitas difiere visiblemente.

Para apreciar las diferencias punto a punto entre los métodos se grafica también un intervalo acotado de las señales. Se puede ver nuevamente que el resultado del método de diferenciación en frecuencia es igual a la derivada analítica.

In [ ]:
a = int(0.2*fs)
b = a + 30
fig = go.Figure()
fig.add_trace(go.Scatter(x=n[a:b], y=x2p[a:b], mode='lines+markers', name='Analítico'))
fig.add_trace(go.Scatter(x=n[a:b], y=x2p_dif[a:b], mode='lines+markers', name='Dif. finitas'))
fig.add_trace(go.Scatter(x=n[a:b], y=x2p_fourier[a:b], mode='lines+markers', name='Fourier'))
fig.update_layout(
    title="x2'[n] según metodo de diferenciación - vista cercana",
    xaxis_title='Tiempo [s]',
    yaxis_title='Amplitud')
fig.show()

Nota: Entiendo que utilizar un gráfico tipo 'line' no es representativo de señales discretas. No encontré en la libreria Plotly un gráfico tipo 'stem', y graficar solamente los 'markers' o puntos individuales es visualmente confuso. Sin embargo utilizé Plotly por consistencia y por las ventajas que sigue brindando.

En caso de que se prefiera se grafican las señales individualmente.

In [ ]:
fig = make_subplots(rows=3, cols=1,
    subplot_titles=('Analítico', 'Diferencias finitas', 'Diferenciación en frecuencia'))
fig.add_trace(go.Scatter(x=n, y=x2p, name='Analítico'), row=1, col=1)
fig.add_trace(go.Scatter(x=n, y=x2p_dif, name='Dif. finitas'), row=2, col=1)
fig.add_trace(go.Scatter(x=n, y=x2p_fourier, name='Fourier'), row=3, col=1)
fig.update_yaxes(title='Amplitud')
fig.update_xaxes(title='Tiempo [s]', row=3)
fig.update_layout(
    title="x2'[n] según metodo de diferenciación",
    showlegend=False)
fig.show()